# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке.

## 1. Откройте и изучите файл

In [1]:
# Импортируем библиотеки
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.shape

(3214, 5)

In [5]:
df.duplicated().sum()

0

In [6]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [8]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


## 2. Разбейте данные на выборки

#### Разобъем выборки на train(80%) и тест(20%)

In [9]:
train, test = train_test_split(df, test_size=0.4, random_state=12345)

In [10]:
features_1 = test.drop('is_ultra', axis=1)
target_1 = test['is_ultra']

#### Разобъем выборки на val(20%) и test(20%)

In [11]:
features_val, features_test, target_val, target_test = train_test_split(features_1, target_1, test_size=0.5, random_state=12345)

In [12]:
features_train = train.drop('is_ultra', axis=1)
target_train = train['is_ultra']

features_train, target_train - обучающая выборка  
features_val, target_val - валидационная выборка  
features_test, target_test - тестовая выборка  

## 3. Исследуйте модели

#### Tree classifier

In [13]:
best_accuracy = 0
best_depth = 0
for depth in range(1,16):
    model = DecisionTreeClassifier(max_depth=depth, random_state=412)
    model.fit(features_train, target_train)
    accuracy = model.score(features_val, target_val)
    if accuracy > best_accuracy:
        # print("Глубина дерева:", depth,"    Качество:" ,accuracy)
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева:", best_depth, "    Качество:", best_accuracy)

Глубина дерева: 3     Качество: 0.7853810264385692


#### Logistic Regression

In [14]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_val, target_val)
print("Качество:", accuracy)

Качество: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### Randomforest

In [15]:
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=412)
            model.fit(features_train, target_train)
            accuracy = model.score(features_val, target_val)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "    Количество деревьев:", best_est, "   мин. объектов в узле:", best_sample, "  Качество:", best_accuracy)

Глубина дерева: 7     Количество деревьев: 40    мин. объектов в узле: 3   Качество: 0.8040435458786936


## 4. Проверьте модель на тестовой выборке

#### Tree classifier

In [16]:
model = DecisionTreeClassifier(max_depth=8, random_state=412)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7962674961119751


#### Logistic Regression

In [17]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7402799377916018


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### Randomforest

In [18]:
model = RandomForestClassifier(max_depth=7, n_estimators=40, min_samples_leaf=3, random_state=412)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7947122861586314


> Наилучшее качество (0.794) достигается при использовании randomforest с гиперпараметрами 
> `max_depth=7` , `n_estimators=40` ,  `min_samples_leaf=3`

## 5. Проверьте модели на адекватность

In [19]:
dmb = DummyClassifier(strategy="stratified" ,random_state=412)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество наивного прогноза :", acc)
print("Качество нашего прогноза :", accuracy)

Качество наивного прогноза : 0.5707620528771384
Качество нашего прогноза : 0.7947122861586314


In [20]:
dmb = DummyClassifier(strategy='most_frequent', random_state=412)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество наивного прогноза :", acc)
print("Качество нашего прогноза :", accuracy)

Качество наивного прогноза : 0.6842923794712286
Качество нашего прогноза : 0.7947122861586314


In [21]:
dmb = DummyClassifier(strategy='uniform', random_state=412)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество наивного прогноза :", acc)
print("Качество нашего прогноза :", accuracy)

Качество наивного прогноза : 0.48989113530326595
Качество нашего прогноза : 0.7947122861586314


# Вывод: 

> #### Проверены три модели: деревья решений, логистическая регрессия, случайный лес.  
> #### Наилучшее качество (0.794) достигается при использовании randomforest с гиперпараметрами 
> `max_depth=7` , `n_estimators=40` , ` min_samples_leaf=3`